# Project Objective

The goal of this project is to:
- Calculate the 99% 1-day and 10-day Value at Risk (VaR) for individual assets and a portfolio using the historical simulation method.
- Perform backtesting of the VaR estimates using Kupiec's proportion of failures test and Christoffersen's independence test.
- Calculate the regulatory capital requirement according to the Basel II framework.

This project demonstrates risk management techniques commonly used in financial institutions.


In [34]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.stats import chi2

# Load stock prices
mbank = pd.read_csv("C:/Users/Nico/Desktop/python/dane/mbk_d.csv", index_col=0, header=0, parse_dates=True)
kety = pd.read_csv("C:/Users/Nico/Desktop/python/dane/kty_d.csv", index_col=0, header=0, parse_dates=True)
pepco = pd.read_csv("C:/Users/Nico/Desktop/python/dane/pco_d.csv", index_col=0, header=0, parse_dates=True)

# Select the "Close" prices up to a certain date
kety = kety.loc[:'2024-12-31', 'Zamkniecie']
pepco = pepco.loc[:'2024-12-31', 'Zamkniecie']
mbank = mbank.loc[:'2024-12-31', 'Zamkniecie']

# Combine all into a single DataFrame
prices = pd.concat([kety, pepco, mbank], axis=1).dropna()
prices.columns = ['KETY', 'PEPCO', 'MBANK']
prices.index.name = None
prices.tail(3)

,KETY,PEPCO,MBANK
2024-12-23,685.0,16.0879,553.6
2024-12-27,679.5,16.1913,549.0
2024-12-30,682.5,16.1716,547.2


# 1. Importing Libraries and Loading Data
In this section, we import the necessary Python libraries and load daily stock price data for three companies: KETY, PEPCO, and MBANK. We then select the closing prices up to December 31, 2024, and combine the data into a single DataFrame for further analysis.


In [35]:
# Calculate daily log returns
log_returns = np.log(prices / prices.shift(1)).dropna()

# Calculate 1-day 99% rolling VaR over a 250-day window
var_1d_99 = log_returns.rolling(window=250).quantile(0.01).dropna()

# Scale the 1-day VaR to 10 days assuming sqrt(time) rule
var_10d_99 = var_1d_99 * (10 ** 0.5)

# Define an equally weighted portfolio
portfolio_weights = np.array([1/3, 1/3, 1/3])
portfolio_log_returns = log_returns.dot(portfolio_weights)

# Calculate portfolio VaR
portfolio_var_1d_99 = portfolio_log_returns.rolling(window=250).quantile(0.01).dropna()
portfolio_var_10d_99 = portfolio_var_1d_99 * (10 ** 0.5)

# 2. Calculating Log Returns and Value at Risk (VaR)
We calculate the daily log returns for each asset and the portfolio. Using a 250-day rolling window, we estimate the 1-day 99% Value at Risk (VaR). We also compute a 10-day VaR using the square root of time rule, assuming returns are independent and identically distributed.


In [36]:
# Define Kupiec Test function
def kupiec_test(exceptions, confidence_level, alpha=0.05):
    n1 = exceptions.sum()
    n = len(exceptions)
    n0 = n - n1
    q_hat = n1 / n
    q_0 = 1 - confidence_level

    LR_POF = -2 * (
        np.log((1 - q_0)**n0 * q_0**n1) -
        np.log((1 - q_hat)**n0 * q_hat**n1)
    )
    chi2_crit = chi2.ppf(1 - alpha, df=1)
    return LR_POF, chi2_crit, n1


# 3. Implementing the Kupiec Test (Proportion of Failures Test)
The Kupiec Test evaluates whether the number of observed VaR breaches is consistent with the expected number, under the null hypothesis of correct coverage. We implement this statistical test and calculate the likelihood ratio.


In [37]:
# Define Christoffersen Test function
def christoffersen_test(exceptions, alpha=0.05):
    T = exceptions.values
    n00 = n01 = n10 = n11 = 0

    for t in range(1, len(T)):
        if T[t - 1] == 0 and T[t] == 0:
            n00 += 1
        elif T[t - 1] == 0 and T[t] == 1:
            n01 += 1
        elif T[t - 1] == 1 and T[t] == 0:
            n10 += 1
        elif T[t - 1] == 1 and T[t] == 1:
            n11 += 1

    pi01 = n01 / (n00 + n01) if (n00 + n01) > 0 else 0
    pi11 = n11 / (n10 + n11) if (n10 + n11) > 0 else 0
    pi = (n01 + n11) / (n00 + n01 + n10 + n11)

    logL_indep = 0
    if pi not in [0, 1]:
        logL_indep = (n01 + n11) * np.log(pi) + (n00 + n10) * np.log(1 - pi)

    logL_dep = 0
    if pi01 not in [0, 1] and pi11 not in [0, 1]:
        logL_dep = (
            n00 * np.log(1 - pi01) + n01 * np.log(pi01) +
            n10 * np.log(1 - pi11) + n11 * np.log(pi11)
        )

    LR_CC = -2 * (logL_indep - logL_dep)
    chi2_crit = chi2.ppf(1 - alpha, df=1)

    return LR_CC, chi2_crit


# 4. Implementing the Christoffersen Test (Independence Test)
The Christoffersen Test examines whether VaR breaches are independently distributed over time. A clustering of breaches would indicate a failure of the model's assumptions. This function calculates the likelihood ratio for independence.


In [38]:
# Set parameters
sample_size = 250
test_moment = '2024-12-30'
confidence_level = 0.99
alpha = 0.05
results = []

# List of assets and portfolio
assets = list(log_returns.columns) + ["PORTFEL"]

# Perform backtesting
for asset in assets:
    if asset == "PORTFEL":
        returns = portfolio_log_returns
        var = portfolio_var_1d_99
    else:
        returns = log_returns[asset]
        var = var_1d_99[asset]

    if test_moment not in var.index:
        raise ValueError(f"No data for {test_moment} for {asset}")

    dates = var.loc[:test_moment].index[-sample_size:]
    ret_sample = returns.loc[dates]
    var_sample = var.loc[dates]
    exceptions = (ret_sample < var_sample).astype(int)

    LR_POF, chi2_pof, n_exceptions = kupiec_test(exceptions, confidence_level, alpha)
    LR_CC, chi2_cc = christoffersen_test(exceptions, alpha)

    results.append({
        "Asset": asset,
        "Number of Breaches": int(n_exceptions),
        "LR_POF": LR_POF,
        "chi2_crit_POF": chi2_pof,
        "LR_CC": LR_CC,
        "chi2_crit_CC": chi2_cc,
        "Decision_POF": "Reject H0" if LR_POF > chi2_pof else "Do not reject H0",
        "Decision_CC": "Reject H0" if LR_CC > chi2_cc else "Do not reject H0"
    })

# Convert results to DataFrame
test_results = pd.DataFrame(results)
test_results


,Asset,Number of Breaches,LR_POF,chi2_crit_POF,LR_CC,chi2_crit_CC,Decision_POF,Decision_CC
0,KETY,6,3.555355,3.841459,2.423191,3.841459,Do not reject H0,Do not reject H0
1,PEPCO,1,1.176491,3.841459,13.030884,3.841459,Do not reject H0,Reject H0
2,MBANK,6,3.555355,3.841459,56.562567,3.841459,Do not reject H0,Reject H0
3,PORTFEL,2,0.108435,3.841459,23.281115,3.841459,Do not reject H0,Reject H0


# 5. Performing Backtesting for Individual Assets and Portfolio

We apply both the Kupiec and Christoffersen tests to each asset and to the portfolio.  
The goal is to evaluate the predictive performance of the Value at Risk (VaR) model by checking:

- **Coverage accuracy** (Kupiec Test — Proportion of Failures)
- **Independence of breaches** (Christoffersen Test)

---

## Interpretation of Backtesting Results:

- **Kupiec Test (POF Test – Proportion of Failures):**  
  For all assets and the portfolio, the Kupiec test does **not reject the null hypothesis**.  
  This means that the number of breaches observed is statistically consistent with the expected number based on the 99% VaR model.  
  In other words, the VaR model achieves the correct overall coverage level.

- **Christoffersen Test (Independence Test):**  
  The Christoffersen test shows mixed results:
  - For **KETY**, we do **not reject the null hypothesis**, suggesting that breaches occur independently over time.
  - For **PEPCO**, **MBANK**, and the **Portfolio**, we **reject the null hypothesis**, indicating that breaches are not independent.  
    This suggests that when a breach happens, there is a higher probability that another breach will occur shortly after (clustering of exceptions).  
    It may imply periods of higher volatility not properly captured by the VaR model.

- **General Conclusion:**  
  While the VaR model provides acceptable **coverage** for all individual assets and the portfolio (correct number of exceptions), it **fails to fully capture the time dynamics** of risk for PEPCO, MBANK, and the Portfolio.  
  In practice, this could indicate that the model underestimates the persistence of risk during volatile periods.

---


In [39]:
# Define function to assign multiplier based on number of exceptions
def assign_multiplier(x):
    if x <= 4:
        return 3
    elif 5 <= x < 9:
        return 3 + 0.2 * (x - 4)
    else:
        return 4

# Define function to calculate capital requirement
def calculate_capital_requirement(var_test_day, average_var, multiplier):
    max_value = np.maximum(var_test_day, average_var * multiplier)
    return -np.abs(max_value)

# Parameters
sample_size = 59
test_moment = '2024-12-30'
results = []

# Calculate regulatory capital
for asset in assets:
    if asset == "PORTFEL":
        returns = portfolio_log_returns
        var = portfolio_var_1d_99
    else:
        returns = log_returns[asset]
        var = var_1d_99[asset]

    if test_moment not in var.index:
        raise ValueError(f"No data for {test_moment} for {asset}")

    dates = var.loc[:test_moment].index[-sample_size-1:-1]
    ret_sample = returns.loc[dates]
    var_sample = var.loc[dates]
    exceptions = (ret_sample < var_sample).astype(int)

    average_var = var_sample.mean()
    n_exceptions = exceptions.sum()
    multiplier = assign_multiplier(n_exceptions)
    var_test_day = var.loc[test_moment]
    capital_requirement = calculate_capital_requirement(var_test_day, average_var, multiplier)

    results.append({
        "Asset": asset,
        "Number of Breaches": int(n_exceptions),
        "Capital Requirement": capital_requirement
    })

# Convert results to DataFrame
capital_requirements = pd.DataFrame(results)
capital_requirements


,Asset,Number of Breaches,Capital Requirement
0,KETY,3,-0.041091
1,PEPCO,0,-0.067565
2,MBANK,2,-0.049465
3,PORTFEL,2,-0.037535


# 6. Calculating Basel Regulatory Capital Requirements
Based on the number of VaR breaches over the last 59 days, we assign a multiplier following Basel II rules. The regulatory capital requirement is calculated by comparing the latest VaR estimate with the scaled average VaR and applying the appropriate multiplier.

- **Number of Breaches:** All assets and the portfolio observed fewer than 5 breaches over the last 59 days. Therefore, the Basel multiplier remains at the **minimum level (3)** for all cases.
- **Capital Requirements:**  
  - The capital requirements are relatively moderate, reflecting stable risk profiles over the observation window.
  - **PEPCO** has the largest absolute capital requirement (higher in magnitude), suggesting that even though there were no breaches, its overall risk exposure (VaR) was comparatively higher.
  - **The Portfolio** shows a smaller requirement compared to individual assets, indicating the benefit of diversification.


In [40]:
prices.to_csv("prices.csv")